In [23]:
import torch
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler

data, y = torch.load('data/data_v2.pkl')
train_data, test_data = data[:len(y)], data[len(y):]

np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(train_data, y, train_size=.8, )

In [26]:
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import BaggingClassifier, StackingClassifier
import numpy as np

std = StandardScaler()
bagging_svm = BaggingClassifier(base_estimator=SVC(probability=True))
bagging_svm.set_params(
    **{'n_estimators': 99, 'base_estimator__C': 36.06241178040971, 'base_estimator__gamma': 0.004735786412937079}
)
lr = LogisticRegression()
adaboost = AdaBoostClassifier(base_estimator=SVC(probability=True))
adaboost_tree = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
adaboost_tree.set_params(**{'n_estimators': 349, 'learning_rate': 0.01, 'base_estimator__max_depth': 36,
                            'base_estimator__min_samples_split': 189, 'base_estimator__min_samples_leaf': 1,
                            'base_estimator__max_features': 25, 'base_estimator__max_leaf_nodes': 471}
                         )
adaboost.set_params(
    **{'n_estimators': 65, 'learning_rate': 0.03694467169486954, 'base_estimator__C': 0.28335389417624707,
       'base_estimator__gamma': 0.007507640146291138}
)
gdbt = GradientBoostingClassifier()
gdbt.set_params(
    **{'n_estimators': 300, 'learning_rate': 0.06504234821394586, 'max_depth': 3, 'min_samples_split': 2,
       'min_samples_leaf': 1, 'max_features': 14, 'max_leaf_nodes': 891}

)
knn = KNeighborsClassifier()
knn.set_params(**{'n_neighbors': 6, 'p': 2, 'weights': 'uniform'}
               )
rf = RandomForestClassifier()
rf.set_params(
    **{'n_estimators': 276, 'max_depth': 99, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 21,
       'max_leaf_nodes': 31}
)

svc = SVC(probability=True)
svc.set_params(**{'C': 1.0121047403763643, 'gamma': 0.17953963922087096}
               )
tree = DecisionTreeClassifier()
tree.set_params(
    **{'max_depth': 3, 'min_samples_split': 103, 'min_samples_leaf': 1, 'max_features': 24}

)

voting_model = StackingClassifier([('lr', lr), ('ada', adaboost), ('gdbt', gdbt),
                                   ('knn', knn), ('svc', svc), ('tree', tree),
                                   ('bagging_svm', bagging_svm), ], n_jobs=-1,
                                  verbose=1, cv=10, final_estimator=DecisionTreeRegressor())
pipe = Pipeline([
    ('std', std),
    ('voting', voting_model)
])

pipe.fit(X_train, y_train)

ValueError: 'final_estimator' parameter should be a classifier. Got DecisionTreeRegressor()

In [25]:
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.7471910112359551, 0.7374301675977654)

In [15]:
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.8792134831460674, 0.8268156424581006)

In [21]:
pipe.fit(train_data, y)
pipe.score(train_data, y)

0.8832772166105499

In [22]:
import pandas as pd

best_model = pipe
y_test = best_model.predict(test_data.values)
result = pd.DataFrame(y_test, index=test_data.index, columns=['Survived'], )
result.to_csv('stacking.csv')

d:\project\python\.venv\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
